In [ ]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

# Configuración de la cámara
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Ancho de la ventana
cap.set(4, 720)   # Alto de la ventana

# Detector de manos
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Variables para dibujar
drawing = False  # Si está dibujando o no
points_dict = {}  # Diccionario para almacenar los puntos asociados a colores, grosores y modos de pintura

# Lista de colores y grosores disponibles
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255)]  # Rojo, verde, azul, amarillo, magenta, cian
thicknesses = [5, 10, 15, 20]  # Lista de grosores disponibles
paint_modes = ['normal', 'highlighter', 'oil']  # Modos de pintura
current_color = colors[0]
current_thickness = thicknesses[1]
current_paint_mode = paint_modes[0]

# Inicializar diccionario para el primer color, grosor y modo de pintura
if (current_color, current_thickness, current_paint_mode) not in points_dict:
    points_dict[(current_color, current_thickness, current_paint_mode)] = []

# Banderas para las interfaces
color_interface_active = False
thickness_interface_active = False
paint_mode_interface_active = False

# Función para interpolar los puntos y dibujar líneas continuas
def interpolate_points(p1, p2):
    points = []
    x1, y1 = p1
    x2, y2 = p2
    distance = int(((x2 - x1)**2 + (y2 - y1)**2)**0.5)
    if distance == 0:
        return [p1]
    for i in range(1, distance + 1):
        t = i / distance
        x = int(x1 + (x2 - x1) * t)
        y = int(y1 + (y2 - y1) * t)
        if (x, y) not in points:
            points.append((x, y))
    return points





def draw_oil_painting(img, points, thickness, color):
    overlay = img.copy()  # Capa de superposición para pintar
    mask = np.zeros_like(img, dtype=np.uint8)  # Máscara para las áreas pintadas

    for point in points:
        if point is None:
            continue
        
        # Variación del color para simular textura de óleo (bordes suaves y mezcla de colores)
        for offset in range(1, 3):  # Varias capas de bordes
            blur_color = tuple(min(255, max(0, c + np.random.randint(-20, 20))) for c in color)  # Variación de color
            cv2.circle(overlay, point, thickness // 2 + offset, blur_color, 1)  # Contornos difusos

        # Crear la máscara para las áreas pintadas
        cv2.circle(mask, point, thickness // 2, (255, 255, 255), -1)

    # Aplicar desenfoque solo en las áreas de la máscara
    blurred_overlay = cv2.GaussianBlur(overlay, (thickness * 2 + 1, thickness * 2 + 1), thickness / 2)
    img[mask[:, :, 0] > 0] = blurred_overlay[mask[:, :, 0] > 0]  # Actualizar solo las áreas pintadas

    # Superponer la textura de óleo solo en las áreas pintadas
    texture = np.random.randint(150, 255, img.shape[:2], dtype=np.uint8)  # Crear textura base
    texture = cv2.GaussianBlur(texture, (21, 21), 10)  # Suavizar textura
    texture_overlay = cv2.merge([texture, texture, texture])

    # Combinar la textura con las áreas pintadas
    alpha_texture = 0.2  # Ajustar la intensidad de la textura
    img[mask[:, :, 0] > 0] = cv2.addWeighted(
        texture_overlay[mask[:, :, 0] > 0],
        alpha_texture,
        img[mask[:, :, 0] > 0],
        1 - alpha_texture,
        0
    )

def draw_paint_mode(img, points, mode, thickness, color):
    if mode == 'normal':
        for point in points:
            if point is None:
                continue
            cv2.circle(img, point, thickness // 2, color, -1)

    elif mode == 'highlighter':
        overlay = img.copy()
        bright_color = tuple(min(255, int(c * 5.0)) for c in color)  # Hacer el color más brillante
        for point in points:
            if point is None:
                continue
            cv2.circle(overlay, point, thickness // 2, bright_color, -1)
        alpha = 0.6  # Transparencia para subrayador
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    elif mode == 'oil':
        draw_oil_painting(img, points, thickness, color)



# Bucle principal
while True:
    # Leer el frame de la cámara
    success, img = cap.read()
    if not success:
        break

    # Detectar manos
    hands, img = detector.findHands(img)

    # Dibujar las líneas entre puntos en la pantalla
    for (color, thickness, mode), points in points_dict.items():
        draw_paint_mode(img, points, mode, thickness, color)

    if hands:
        hand = hands[0]  # Solo la primera mano detectada
        lmList = hand["lmList"]  # Lista de posiciones (landmarks)

        # Coordenadas del dedo índice (punta)
        x_index, y_index = lmList[8][:2]  # Índice

        # Coordenadas del pulgar (punta)
        x_thumb, y_thumb = lmList[4][:2]  # Pulgar

        x = (x_index + x_thumb) // 2
        y = (y_index + y_thumb) // 2

        # Distancia entre el índice y el pulgar
        length, _, _ = detector.findDistance(lmList[8][:2], lmList[4][:2], img)

        # Detectar si la mano está cerrada (puño) para borrar
        fingers_down = all(lmList[f][1] > lmList[f - 2][1] for f in [8, 12, 16, 20])
        if fingers_down and not (color_interface_active or thickness_interface_active or paint_mode_interface_active):
            # Dibujar el círculo de borrado
            cv2.circle(img, (x, y), current_thickness, (0, 255, 255), -1)  # Borde amarillo
            cv2.circle(img, (x, y), current_thickness - 2, (0, 0, 0), -1)  # Interior negro
            for (color, thickness, mode), points in points_dict.items():
                points[:] = [point for point in points if point is None or ((x - point[0])**2 + (y - point[1])**2)**0.5 > current_thickness ]

        elif not color_interface_active and not thickness_interface_active and not paint_mode_interface_active and lmList[8][1] < lmList[6][1]:
            # Activar interfaz de color, grosor o modo de pintura según posición del dedo
            if x_index < img.shape[1] // 3:
                color_interface_active = True
            elif x_index < img.shape[1] * 2 // 3:
                thickness_interface_active = True
            else:
                paint_mode_interface_active = True

        elif not (color_interface_active or thickness_interface_active or paint_mode_interface_active):  # Solo permitir dibujar si ninguna interfaz está activa
            if length < 50:
                if (current_color, current_thickness, current_paint_mode) not in points_dict:
                    points_dict[(current_color, current_thickness, current_paint_mode)] = []
                if not drawing:
                    points_dict[(current_color, current_thickness, current_paint_mode)].append(None)
                drawing = True
                if len(points_dict[(current_color, current_thickness, current_paint_mode)]) > 0:
                    last_point = points_dict[(current_color, current_thickness, current_paint_mode)][-1]
                    if last_point is not None:
                        interpolated_points = interpolate_points(last_point, (x, y))
                        points_dict[(current_color, current_thickness, current_paint_mode)].extend(interpolated_points)
                    else:
                        points_dict[(current_color, current_thickness, current_paint_mode)].append((x, y))
            else:
                drawing = False

        # Interfaz de selección de color
        if color_interface_active:
            color_rectangles = []
            num_colors = len(colors)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_colors * rect_width + (num_colors - 1) * spacing)) // 2
            start_y = img.shape[0] - 100

            for i, color in enumerate(colors):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), color, -1)
                color_rectangles.append((color, (x1, y1, rect_width, 50)))

            if length < 50:
                for color, (x1, y1, w, h) in color_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_color = color
                        color_interface_active = False
                        break

        # Interfaz de selección de grosor
        if thickness_interface_active:
            thickness_circles = []
            num_thicknesses = len(thicknesses)
            spacing = 20  # Espacio entre los círculos
            start_x = (img.shape[1] - (num_thicknesses * (max(thicknesses) * 2 + spacing) - spacing)) // 2
            start_y = img.shape[0] - 200

            for i, thickness in enumerate(thicknesses):
                center_x = start_x + i * (thickness * 2 + spacing)  # Ajuste de posición según el grosor
                center_y = start_y
                # Dibujar un círculo con borde negro y relleno con el color actual
                cv2.circle(img, (center_x, center_y), thickness, (0, 0, 0), -1)  # Borde negro
                cv2.circle(img, (center_x, center_y), thickness - 1, current_color, -1)  # Relleno con el color actual
                thickness_circles.append((thickness, (center_x, center_y, thickness)))

            # Detectar si el clic está dentro de algún círculo
            if length < 50:
                for thickness, (center_x, center_y, radius) in thickness_circles:
                    if (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2:  # Verificar si el clic está dentro del círculo
                        current_thickness = thickness
                        thickness_interface_active = False
                        break

        # Interfaz de selección de modo de pintura
        if paint_mode_interface_active:
            mode_rectangles = []
            num_modes = len(paint_modes)
            rect_width = 60
            spacing = 20
            start_x = (img.shape[1] - (num_modes * rect_width + (num_modes - 1) * spacing)) // 2
            start_y = img.shape[0] - 300

            for i, mode in enumerate(paint_modes):
                x1, y1 = start_x + i * (rect_width + spacing), start_y
                cv2.rectangle(img, (x1, y1), (x1 + rect_width, y1 + 50), (200, 200, 200), -1)
                cv2.putText(img, f"{mode}", (x1 + 10, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                mode_rectangles.append((mode, (x1, y1, rect_width, 50)))

            if length < 50:
                for mode, (x1, y1, w, h) in mode_rectangles:
                    if x1 <= x <= x1 + w and y1 <= y <= y1 + h:
                        current_paint_mode = mode
                        paint_mode_interface_active = False
                        break

    # Mostrar la imagen resultante
    cv2.imshow("Dibujo", img)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
